In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
from pprint import pprint

# 판매량을 예측할수 있을까?

### 판매량에 영향을 끼칠수 있는 변수는 무었이 있을까?
+ 인지도
+ 흥보상태
+ 노래 평판?
+ 팬
  + 나이
  + 성별
+ 소속사

### 변수들을 수집할 방법은 무었이 있을까?
+ 인지도 : 구글 트렌드

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

response = requests.get("http://www.gaonchart.co.kr/main/section/chart/online.gaon?nationGbn=T&serviceGbn=S1020&targetTime=12&hitYear=2011&termGbn=week" ) 
soup = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')

titles = soup.select('tr p')

In [7]:
a = []
for i in range(0, 500, 5):
    for j in range(5):
        a.append(titles[i+j].text)

In [8]:
columns = ['Title', 'Artist/album', 'Download', 'Production', 'Distribution']
df = pd.DataFrame(np.array(a).reshape((100,5)), index=range(1,100+1), columns = columns)
df.insert(loc=list(df).index('Title')+1 ,column='Artist', value=df.loc[:,'Artist/album'].apply(lambda x: x.split('|')[0]))
df.insert(loc=list(df).index('Artist')+1 ,column='album', value=df.loc[:,'Artist/album'].apply(lambda x: x.split('|')[1]))
df.__delitem__('Artist/album')

In [9]:
df

,Title,Artist,album,Download,Production,Distribution
1,미친거니 (Feat. 방용국),송지은,미친거니 (Digital Single),"330,488",TS엔터테인먼트,로엔엔터테인먼트
2,가질 수 없는 너,현빈,"현빈, 가질 수 없는 너","292,956",예술의 봄,SBS 콘텐츠허브
3,Tonight,빅뱅,빅뱅 미니앨범 4집,"271,224",YG Entertainment,지니뮤직
4,바람이 분다,이소라,6집 눈썹달,"235,286",아인스디지탈,IS MUSIC
5,Cafe,빅뱅,빅뱅 미니앨범 4집,"196,849",YG Entertainment,지니뮤직
6,기다리고 싶어,거미,마이더스 OST Part 2,"193,584",포니캐년코리아,포니캐년코리아
7,가슴이 뛴다,케이윌 (K.Will),가슴이 뛴다,"190,675",스타쉽엔터테인먼트,로엔엔터테인먼트
8,나만 몰랐던 이야기,아이유 (IU),Real+,"189,830",로엔엔터테인먼트,로엔엔터테인먼트
9,What Is Right,빅뱅,빅뱅 미니앨범 4집,"186,385",YG Entertainment,지니뮤직
10,꿈에,박정현,박정현 Op.4,"179,750",T Entertainment,NHN벅스
